# Exercise 5

## Helper functions

In [1]:
import numpy as np
import cv2 
import os
import scipy
import matplotlib.pyplot as plt
import matplotlib

def load_im(path : str) -> np.ndarray:
    
    im = cv2.imread(path)[:, :, ::-1]
    im = im.astype(np.float64) / 255
    
    return im


def pi(points : np.ndarray) -> np.ndarray:
    """
        Converts from homogeneous to inhomogeneous coordinates
    """
    p = points[:-1]/points[-1]
    
    return p


def piInv(points : np.ndarray) -> np.ndarray:
    """
        Converts from inhomogeneous to homogeneous coordinates
    """
    
    # Gets the amount of points by using shape
    _, num_points = points.shape
    
    # Stacks the scale s at the bottom of the matrix
    ph = np.vstack((points, np.ones(num_points)))
    
    return ph


def projectPoints(K, Rt, Q):
    
    Q_hom = piInv(Q)
    points = K @ Rt @ Q_hom
    points_inhom = pi(points)
    
    return points_inhom


def hest(q1, q2) -> np.ndarray:
    """
        Takes two points in 2D and returns the estimated homography matrix.
    """
    
    if len(q1) != len(q2):
        raise ValueError("There must be an equal amount of points in the two sets!")
    
    Bi = []
    for i in range(q1.shape[1]):
        qi = q1[:,i]   # <-- getting the first column
        
        # Creating that weird qx matrix for the Kronecker product
        q1x = np.array(
            [[0,        -1, qi[1]],
             [1,        0, -qi[0]],
             [-qi[1], qi[0], 0]]
        )
        
        q2t_hom = q2[:, i].reshape(-1, 1) # <-- getting the first column and reshaping does for dim: (1, ) -> (1,1)
        Bi.append(np.kron(q2t_hom.T, q1x)) # <-- formula follows that of week 2, slide 56
        # print(np.kron(q2t_hom.T, q1x).shape)
       
    B = np.concatenate(Bi, axis=0)
    
    # Some TA prooved that it was unneseccary to find their dot product
    #BtB = B.T @ B
    V, Lambda, Vt = np.linalg.svd(B)
    Ht = Vt[-1, :]
    
    Ht = np.reshape(Ht, (3, 3))
    H = Ht.T
    
    return H
    
    
def crossOp(p : np.ndarray) -> np.ndarray:
    """
        One of Them weird functions. It takes in a 3D vector and then returns
        some gnarly matrix.
    """
    p = p.flatten()
    if p.size != 3:
        raise Exception("Invalid input, vector must be exactly 3D.")
    
    x, y, z = p
    px = np.array(
        [[0, -z, y],
         [z, 0, -x],
         [-y, x, 0]]
    )
    
    return px


def computeFundamentalMatrix(K1 : np.ndarray, K2 : np.ndarray, R2 : np.ndarray, t2 : np.ndarray) -> np.ndarray:
    """
        Computing the fundamental matrix between two camera matrices K1 & K2.
    """
    t2x = crossOp(t2)

    E = t2x @ R2

    K1inv = np.linalg.inv(K1)
    K2inv = np.linalg.inv(K2)

    F = K1inv.T @ E @ K2inv
    
    return F


def fancyRotate(theta_x, theta_y, theta_z):
    """
        Does the rotation matrix that we have seen a few times.
        E.g. Exercises week 4, eq(12).
    """
    from scipy.spatial.transform import Rotation
    
    R = Rotation.from_euler("xyz", [theta_x, theta_y, theta_z]).as_matrix()
    
    return R




## Initial setup
Its propaply good to remember that:
$$
    \begin{align*}
        \pmb{p}_h &= \pmb{K} \pmb{P}_{cam}
        \\
        &=  \pmb{K} \left[ \pmb{R} \pmb{t} \right] \pmb{P}_h
    \end{align*}
$$

Where:
$$
    \begin{equation*}
        \mathcal{P} = \pmb{K} \left[ \pmb{R} \pmb{t} \right]
    \end{equation*}
$$

As per usual we have some rotations and translations. They are defined as follows:

$$
    \begin{gather*}
        \pmb{R}_1 = \pmb{R}_2 = \pmb{I},
    \end{gather*}
$$

$$
    \begin{align*}
        \pmb{t}_1 &=
        \begin{bmatrix}
            0 & 0 & 1
        \end{bmatrix}^T
        \\
        \pmb{t}_2 &=
        \begin{bmatrix}
            0 & 0 & 20
        \end{bmatrix}^T
    \end{align*}
$$

$$
    \begin{gather*}
        \pmb{K}_1 = \pmb{K}_2 = 
        \begin{bmatrix}
            700 & 0     & 600 \\
            0   & 700   & 400 \\
            0   & 0     & 1
        \end{bmatrix}.
    \end{gather*}
$$

Where both cameras observe the point:
$$
    \begin{gather*}
        \pmb{Q} = 
        \begin{bmatrix}
            1 & 1 & 0
        \end{bmatrix}^T.
    \end{gather*}
$$

In [2]:
import copy

R1 = np.eye(3)
R2 = copy.deepcopy(R1)

t1 = np.array([0, 0, 1]).reshape(-1, 1)
t2 = np.array([0, 0, 20]).reshape(-1, 1)

K1 = np.array(
    [[700,  0,      600],
     [0,    700,    400],
     [0,    0,      1]]
)

K2 = copy.deepcopy(K1)

Q = np.array([1, 1, 0]).reshape(-1, 1)

## Ex. 5.1

What are the projectiopn matrices $\pmb{P}_1$ and $\pmb{P}_2$?

What is the projection of $\pmb{Q}$ in cameras one and two ($\pmb{q}_1$ and $\pmb{q}_2$)?


### Response

In [3]:
Rt1 = np.concatenate((R1, t1), axis=1)
Rt2 = np.concatenate((R2, t2), axis=1)


P1 = K1 @ Rt1
P2 = K2 @ Rt2

qs1 = projectPoints(K1, Rt1, Q)
qs2 = projectPoints(K2, Rt2, Q)

print(f"Yields projection matrices: \nP1\n{P1} \nP2\n{P2}\n")
print(f"\nand the projected points: \nqs1\n{qs1} \nqs2\n{qs2}\n")

Yields projection matrices: 
P1
[[700.   0. 600. 600.]
 [  0. 700. 400. 400.]
 [  0.   0.   1.   1.]] 
P2
[[7.0e+02 0.0e+00 6.0e+02 1.2e+04]
 [0.0e+00 7.0e+02 4.0e+02 8.0e+03]
 [0.0e+00 0.0e+00 1.0e+00 2.0e+01]]


and the projected points: 
qs1
[[1300.]
 [1100.]] 
qs2
[[635.]
 [435.]]



## Ex. 5.2